In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('datasets/tmdb_5000_movies.csv')
df = pd.read_csv('datasets/tmdb_5000_credits.csv')

In [3]:
# df.info
df.columns = ['id', 'tittle', 'cast', 'crew']
movies = movies.merge(df, on='id')

In [4]:
 movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [6]:
movies['overview'].head(3)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
Name: overview, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# removing english stop word like a, and , the 
tfidf = TfidfVectorizer(analyzer = 'word',stop_words = 'english')

#NaN -> ''
movies['overview'] = movies['overview'].fillna('')

tfidf_matrix =  tfidf.fit_transform(movies['overview'])

tfidf_matrix.shape

(4803, 20978)

from above shape we can see that there are 20978 differrent words are used to descibe a 4803 movies.

Now, we will find similarity score of this matrix.

As we have a TF_IDF vectorizer, calculating directly a dot product will give us a cosine similarity. here we are using cosine similarity score since it is relatively easy and fast to calculate.


In [8]:
from sklearn.metrics.pairwise import linear_kernel

cosin_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosin_sim[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [9]:
# reverse map indices and movie titles
index_of_movies = pd.Series(movies.index, index=movies['title']).drop_duplicates()
index_of_movies

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [10]:
def get_recommendations(title, cosin_sim=cosin_sim):
    idx = index_of_movies[title]
    
    sim_scores = list(enumerate(cosin_sim[idx]))
    # sorting of moviesidx based on similarity score
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    # get top 10 of sorted 
    sim_scores = sim_scores[1:31]
    
    movies_idx = [i[0] for i in sim_scores]
    
    return movies['title'].iloc[movies_idx]


In [11]:
get_recommendations('The Godfather').head(10)

2731     The Godfather: Part II
1873                 Blood Ties
867     The Godfather: Part III
3727                 Easy Money
3623                       Made
3125                     Eulogy
3896                   Sinister
4506            The Maid's Room
3783                        Joe
2244      The Cold Light of Day
Name: title, dtype: object

### Improving the recommender with another metadatas

In [12]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for f in features:
    movies[f] = movies[f].apply(literal_eval)

In [13]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [14]:
# get top 3 elements of list
def get_list(x):
    if isinstance(x, list):
        names = [ i['name'] for i in x]
        
        if len(names)  > 3:
            names = names[:3]
        return names
    return []

In [15]:
movies['director'] = movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for f in features:
    movies[f] = movies[f].apply(get_list)

In [16]:
movies[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton,"[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


In [17]:
# striping
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

In [18]:
features = ['cast', 'keywords', 'director', 'genres']
for f in features:
    movies[f] = movies[f].apply(clean_data)

In [19]:
def create_soup(x):
    return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+x['director']+' '+' '.join(x['genres'])

movies['soup'] = movies.apply(create_soup, axis=1)

In [20]:
# count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(movies['soup'])

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

cosin_sim2 = cosine_similarity(count_matrix, count_matrix)

In [22]:
movies = movies.reset_index()
indx = pd.Series(movies.index, index = movies['title']) 

In [23]:
get_recommendations('The Godfather', cosin_sim2).head(10)

867      The Godfather: Part III
2731      The Godfather: Part II
4638    Amidst the Devil's Wings
2649           The Son of No One
1525              Apocalypse Now
1018             The Cotton Club
1170     The Talented Mr. Ripley
1209               The Rainmaker
1394               Donnie Brasco
1850                    Scarface
Name: title, dtype: object

### collobrative filtering

In [31]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

reader = Reader()
ratings = pd.read_csv('datasets/ratings_small.csv')
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [33]:
ratings['userId'].nunique()

671

In [34]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8939  0.8966  0.8942  0.8933  0.8998  0.8956  0.0024  
Fit time          11.05   10.78   12.07   10.98   11.10   11.20   0.45    
Test time         0.35    0.28    0.39    0.49    0.30    0.36    0.07    


{'test_rmse': array([0.89393531, 0.89661314, 0.89423028, 0.89333985, 0.89977182]),
 'fit_time': (11.051206350326538,
  10.781831979751587,
  12.065945625305176,
  10.983144760131836,
  11.097263097763062),
 'test_time': (0.34861254692077637,
  0.2809569835662842,
  0.38953518867492676,
  0.48636889457702637,
  0.30025434494018555)}

In [35]:
train = data.build_full_trainset()
svd.fit(train)

In [37]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


lets's predict the user 1's rating on the movie id 302.

In [38]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.7033616633036264, details={'was_impossible': False})

here u can see that est=2.62 means that user 1 might be give rating of 2.63 to movieID 302.

In [39]:
svd.predict(2, 5001)

Prediction(uid=2, iid=5001, r_ui=None, est=3.7503047820204842, details={'was_impossible': False})

## Hybrid recommender

Now, lets put our content based and CF based filtering together 


In [40]:
# convert float val to int
def conv_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [41]:
movie_id = pd.read_csv('datasets/links.csv')[['movieId', 'tmdbId']]
movie_id['tmdbId'] = movie_id['tmdbId'].apply(conv_int)
movie_id.columns = ['movieId', 'id']
movie_id = movie_id.merge(movies[['title', 'id']], on='id').set_index('title')
print(movie_id.shape)
movie_id

(4599, 2)


,movieId,id
title,,
Toy Story,1,862.0
GoldenEye,10,710.0
The American President,11,9087.0
Nixon,14,10858.0
Cutthroat Island,15,1408.0
...,...,...
Christmas Mail,172347,76706.0
Good Intentions,172853,34417.0
Growing Up Smith,173857,426469.0


In [42]:
index_map = movie_id.set_index('id')

In [43]:
def recommend_for(userId, title):
    index = index_of_movies[title]
    tmdbId = movie_id.loc[title]['id']
    
    # content based
    sim_scores = list(enumerate(cosin_sim2[int(index)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    movie_indices = [i[0] for i in sim_scores]
    
    mv = movies.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    mv = mv[mv['id'].isin(movie_id['id'])]
    
    # CF
    mv['est'] = mv['id'].apply(lambda x: svd.predict(userId, index_map.loc[x]['movieId']).est)

    mv = mv.sort_values('est', ascending=False)
    
    return mv.head(10)
    

In [44]:
recommend_for(4, 'The Avengers')

,title,vote_count,vote_average,id,est
101,X-Men: First Class,5181,7.1,49538,4.637144
85,Captain America: The Winter Soldier,5764,7.6,100402,4.631101
68,Iron Man,8776,7.4,1726,4.533436
1294,Serenity,1264,7.4,16320,4.502846
7,Avengers: Age of Ultron,6767,7.3,99861,4.493021
182,Ant-Man,5880,7.0,102899,4.478290
94,Guardians of the Galaxy,9742,7.9,118340,4.434046
28,Jurassic World,8662,6.5,135397,4.401922
169,Captain America: The First Avenger,7047,6.6,1771,4.331754
203,X2,3506,6.8,36658,4.318500


In [47]:
recommend_for(2, 'Spectre')

,title,vote_count,vote_average,id,est
205,Sherlock Holmes: A Game of Shadows,3886,7.0,58574,3.901237
354,The Girl with the Dragon Tattoo,2434,7.2,65754,3.785601
356,Sherlock Holmes,5766,7.0,10528,3.734684
134,Mission: Impossible - Rogue Nation,3224,7.1,177677,3.726487
531,The Man from U.N.C.L.E.,2265,7.1,203801,3.709686
1234,The Art of War,135,5.6,11398,3.693684
425,Mission: Impossible,2631,6.7,954,3.639641
277,Casino Royale,3855,7.3,36557,3.564673
29,Skyfall,7604,6.9,37724,3.551825
372,Spy Game,579,6.8,1535,3.513014
